## Installing necessary libraries

In [ ]:
! /opt/conda/bin/python3.7 -m pip install -q --upgrade pip
! pip install -q timm catalyst
! pip install -q --upgrade wandb
! pip install -q pytorch-gradcam

We are going to use `wandb` for tracking our model's performance. If you don't have a `wandb` account, go to [this](wandb.ai) link, create an account using either google or github account. Then go to `wandb.ai/[your_username]` -> `Create New Project`. Give a cute little name to your project. Open your project page. You'll find some line like this:

`wandb login e1da498db2dd649a76a04c6e4743e5a4f95a2ae0`

Copy and paste this line to the next cell.

In [ ]:
! wandb login e1da498db2dd649a76a04c6e4743e5a4f95a2ae0

# Config
This section contains configuration parameters for my classification pipeline.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import cv2
import pandas as pd
import albumentations as A
from albumentations.augmentations.transforms import Equalize, Posterize, Downscale
from albumentations import (
    PadIfNeeded, HorizontalFlip, VerticalFlip, CenterCrop,    
    RandomCrop, Resize, Crop, Compose, HueSaturationValue,
    Transpose, RandomRotate90, ElasticTransform, GridDistortion, 
    OpticalDistortion, RandomSizedCrop, Resize, CenterCrop,
    VerticalFlip, HorizontalFlip, OneOf, CLAHE, Normalize,
    RandomBrightnessContrast, Cutout, RandomGamma, ShiftScaleRotate ,
    GaussNoise, Blur, MotionBlur, GaussianBlur, 
)

SEED = 24
n_epochs = 30
device = 'cuda:0'
data_dir = '../input/cassava-leaf-disease-merged/'
loss_thr = 1e6
img_path = f'{data_dir}/train'
df = pd.read_csv(f'{data_dir}/merged.csv')
df['path'] = df['image_id'].map(lambda x: f"{img_path}/{x}")
encoder_model = 'gluon_seresnext50_32x4d'
fold = 0
model_name= f'SE-Resnext50_fold{fold}' # Will come up with a better name later
model_dir = 'model_dir'
history_dir = 'history_dir'
load_model = False
img_dim = 320
batch_size = 32
accum_step = 1
learning_rate = 2.00e-3
num_workers = 4
mixed_precision = True
patience = 3
balanced_sampler = False
train_aug = A.Compose([A.CenterCrop(p=0.3, height=int(0.7*img_dim), width=int(0.7*img_dim)),
A.augmentations.transforms.RandomCrop(int(0.7*img_dim), int(0.7*img_dim), p=0.3),
A.augmentations.transforms.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
A.augmentations.transforms.Resize(img_dim, img_dim, interpolation=1, always_apply=True, p=0.6),
Cutout(num_holes=8, max_h_size=20, max_w_size=20, fill_value=0, always_apply=False, p=0.2),
A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=0.3),
A.augmentations.transforms.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, always_apply=False, p=0.4),
# A.HorizontalFlip(p=0.5),
A.VerticalFlip(p=0.5),                    
OneOf([
        GaussNoise(var_limit=0.1),
        Blur(),
        GaussianBlur(blur_limit=3),
        # RandomGamma(p=0.7),
        ], p=0.3),
A.HorizontalFlip(p=0.3), Normalize(always_apply=True)],)
val_aug = Compose([Normalize(always_apply=True)])


In [ ]:
df.label.hist()

## Fixing Seed

In [ ]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

# Sample Images

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
fig = plt.figure(figsize=(60, 60))
for class_id in sorted(df['label'].unique()):
    for i, (idx, row) in enumerate(df.loc[df['label'] == class_id].sample(3, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path= row['path']
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (img_dim, img_dim))
        plt.imshow(image)
        ax.set_title('Label: %s Name: %s' % (row['label'], row['image_id']), fontsize=30)

## Data Stratification

In [ ]:
df1 = df[df['source']==2020]
df2 = df[df['source']==2019]

In [ ]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=SEED)
X = df1['path']
y = df1['label']
train_idx = []
val_idx = []

df1['fold'] = np.nan
df2['fold'] = np.nan
df2['fold'] = df2['fold'].map(lambda x: fold)

df1= df1.sample(frac=1, random_state=SEED).reset_index(drop=True)
#split data
for i, (_, test_index) in enumerate(skf.split(X, y)):
    df1.loc[test_index, 'fold'] = i
    
df1['fold'] = df1['fold'].astype('int')

valid_df = df1[df1['fold']==fold]
train_df1 = df1[df1['fold']!=fold]
train_df = pd.concat([train_df1, df2])

## Dataset

In [ ]:
from torch.utils.data import Dataset,DataLoader

def onehot(size, target, one_hot=False):
    if not one_hot: return target
    else:
        vec = torch.zeros(size, dtype=torch.float32)
        vec[target] = 1.
        return vec


class LeafDataset(Dataset):
    def __init__(self, df, dim=256, transforms=None):
        super().__init__()
        self.image_ids = df.path.tolist()
        try:
            self.labels = df.label.tolist()
        except:
            self.labels = None
        self.transforms = transforms
        self.dim = dim
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.dim, self.dim))
        
        if self.transforms is not None:
            aug = self.transforms(image=image)
            image = aug['image'].reshape(self.dim, self.dim, 3).transpose(2, 0, 1)
        else:
            image = image.reshape(self.dim, self.dim, 3).transpose(2, 0, 1)
        if self.labels is not None:
            target = self.labels[idx]
            return image_id, image, onehot(5, target, False)
        else:
            return image_id, image

    def __len__(self):
        return len(self.image_ids)

    def get_labels(self):
        return list(self.labels)

## Model

In [ ]:
from torch import nn
from torch.nn import *
from torch.nn import functional as F
from torchvision import models
import timm

class Resne_t(nn.Module):

    def __init__(self, model_name):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=True)
        self.in_features = self.backbone.fc.in_features
        self.output = nn.Sequential(nn.Linear(self.in_features, 128), nn.Linear(128, 5))

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        try:
            x = self.backbone.act1(x)
        except:
            x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.backbone.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.output(x)
        return x

# Utils

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
import wandb
from functools import partial
from collections import Counter
import gc
import time
import pandas as pd
from torch import optim
from catalyst.data.sampler import BalanceClassSampler

wandb.init(project="quantum_ai_demo")
wandb.run.name= model_name

m_p = mixed_precision
if m_p:
  scaler = torch.cuda.amp.GradScaler() 

np.random.seed(SEED)

train_ds = LeafDataset(train_df, img_dim, train_aug)
if balanced_sampler:
  print('Using Balanced Sampler....')
  train_loader = torch.utils.data.DataLoader(train_ds,batch_size=batch_size, sampler=BalanceClassSampler(labels=train_ds.get_labels(), mode="upsampling"), shuffle=False, num_workers=4)
else:
  train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

val_ds = LeafDataset(valid_df, img_dim, val_aug)
valid_loader = torch.utils.data.DataLoader(
val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

os.makedirs(model_dir, exist_ok=True)
os.makedirs(history_dir, exist_ok=True)

result = pd.DataFrame(columns=['name', 'prediction', 'label', 'difference'])
if os.path.exists(f'{history_dir}/history_{model_name}_{img_dim}.csv'):
    history = pd.read_csv(f'{history_dir}/history_{model_name}_{img_dim}.csv')
else:
    history = pd.DataFrame(columns=['train_loss','train_time','val_loss','val_cat_acc', 'val_time'])

model = Resne_t(encoder_model).to(device)
wandb.watch(model)
criterion = nn.CrossEntropyLoss()

In [ ]:
def save_model(valid_loss, valid_acc, best_valid_loss, best_valid_acc, best_state, savepath):
    if valid_loss<best_valid_loss:
        print(f'Validation loss has decreased from:  {best_valid_loss:.4f} to: {valid_loss:.4f}. Saving checkpoint')
        torch.save(best_state, savepath+'_loss.pth')
        best_valid_loss = valid_loss
    if valid_acc>best_valid_acc:
        print(f'Validation Accuracy score has increased from:  {best_valid_acc:.4f} to: {valid_acc:.4f}. Saving checkpoint')
        torch.save(best_state, savepath + '_acc.pth')
        best_valid_acc = valid_acc
    else:
        torch.save(best_state, savepath + '_last.pth')
    return best_valid_loss, best_valid_acc

## Confusion Matrix

In [ ]:
%matplotlib agg
import itertools
from sklearn.metrics import confusion_matrix
import seaborn as sns


def plot_confusion_matrix(predictions, actual_labels, labels):
    cm = confusion_matrix(predictions, actual_labels, labels)
    # Normalise
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(12,12))
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=labels, yticklabels=labels)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig('confusion_matrix.png')

## Grad-CAM Visualization

In [ ]:
from gradcam import GradCAM, GradCAMpp

def visualize_cam(mask, img, alpha=1.0, beta=0.15):
    
    """
    Courtesy: https://github.com/vickyliin/gradcam_plus_plus-pytorch/blob/master/gradcam/utils.py
    Make heatmap from mask and synthesize GradCAM result image using heatmap and img.
    Args:
        mask (torch.tensor): mask shape of (1, 1, H, W) and each element has value in range [0, 1]
        img (torch.tensor): img shape of (1, 3, H, W) and each pixel value is in range [0, 1]
    Return:
        heatmap (torch.tensor): heatmap img shape of (3, H, W)
        result (torch.tensor): synthesized GradCAM result of same shape with heatmap.
    """
    heatmap = (255 * mask.squeeze()).type(torch.uint8).cpu().numpy()
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_MAGMA)
    heatmap = torch.from_numpy(heatmap).permute(2, 0, 1).float().div(255)
    b, g, r = heatmap.split(1)
    heatmap = torch.cat([r, g, b]) * alpha

    result = heatmap+img.cpu()*beta
    result = result.div(result.max()).squeeze()

    return heatmap, result


def grad_cam_gen(model, img, device = 'cuda'):     
    configs = [dict(model_type='resnet', arch=model, layer_name='layer4_2_se_module_fc2')]
    for config in configs:
        config['arch'].to(device).eval()

    cams = [
    [cls.from_config(**config) for cls in (GradCAM, GradCAMpp)]
        for config in configs]

    for _, gradcam_pp in cams:
        mask_pp, _ = gradcam_pp(img)
        heatmap_pp, result_pp = visualize_cam(mask_pp, img)
        result_pp = result_pp.cpu().numpy()
        #convert image back to Height,Width,Channels
        heatmap_pp = np.transpose(heatmap_pp, (1,2,0))
        result_pp = np.transpose(result_pp, (1,2,0))
        return result_pp

In [ ]:
%matplotlib agg
def plot_heatmap(model):
    fig = plt.figure(figsize=(70, 56))
    for class_id in sorted(valid_df['label'].unique()):
        for i, (idx, row) in enumerate(valid_df.loc[valid_df['label'] == class_id].sample(5, random_state=SEED).iterrows()):
            ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
            path=f"{row['path']}"
            image = cv2.imread(path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (img_dim, img_dim))
            aug = val_aug(image=image)
            image = aug['image'].reshape(img_dim, img_dim, 3).transpose(2, 0, 1)
            image = torch.FloatTensor(image)
            prediction = torch.argmax(model(torch.unsqueeze(image.to(device), dim=0)))
            prediction = prediction.data.cpu().numpy()
            image = grad_cam_gen(model.backbone, torch.unsqueeze(image, dim=0).cuda())
            ax.set_title('Label: %s Prediction: %s' % (row['label'], prediction), fontsize=30)
            plt.imshow(image)
            plt.savefig('heatmap.png')

# Training

In [ ]:
def train_val(epoch, dataloader, optimizer, pretrained=None, train=True, mode='train', record=True):
    global m_p
    global result
    global batch_size
    global accum_step
    t1 = time.time()
    running_loss = 0
    epoch_samples = 0
    pred = []
    lab = []
    if pretrained:
        model.load_state_dict(pretrained)
    if train:
        model.train()
        print("Initiating train phase ...")
    else:
        model.eval()
        print("Initiating val phase ...")
    for idx, (_, img, labels) in enumerate(dataloader):
        with torch.set_grad_enabled(train):
            img = img.to(device, dtype=torch.float32)
            labels = torch.LongTensor(labels).to(device)
            epoch_samples += len(img)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(m_p):
                if m_p:
                    img = img.half()
                else:
                    img = img.float()
                outputs = model(img)

                loss = criterion(outputs, labels).sum()
                running_loss += loss.item()*len(img)
                loss = loss/accum_step
      
                if train:
                     if m_p:
                         scaler.scale(loss).backward()
                         if (idx+1) % accum_step == 0:
                             scaler.step(optimizer)
                             scaler.update() 
                             optimizer.zero_grad()
                     else:
                         loss.backward()
                         if (idx+1) % accum_step == 0:
                             optimizer.step()
                             optimizer.zero_grad()

        elapsed = int(time.time() - t1)
        eta = int(elapsed / (idx+1) * (len(dataloader)-(idx+1)))
        pred.append(torch.argmax(outputs, dim=1).detach().cpu().numpy())
        lab.append(labels.cpu().numpy())
        if train:
            msg = f"Epoch: {epoch} Progress: [{idx}/{len(dataloader)}] loss: {(running_loss/epoch_samples):.4f} Time: {elapsed}s ETA: {eta} s"
        else:
            msg = f'Epoch {epoch} Progress: [{idx}/{len(dataloader)}] loss: {(running_loss/epoch_samples):.4f} Time: {elapsed}s ETA: {eta} s'
        wandb.log({"Train Loss": running_loss/epoch_samples})
        print(msg, end= '\r')
    cat_acc = (np.concatenate(pred)==np.concatenate(lab)).mean()
    history.loc[epoch, f'{mode}_loss'] = running_loss/epoch_samples
    history.loc[epoch, f'{mode}_time'] = elapsed
    if mode=='val' or mode=='test':
        lr_reduce_scheduler.step(cat_acc)
        msg = f'{mode} Loss: {running_loss/epoch_samples:.4f} \n {mode} Categorical Accuracy: {cat_acc:.4f}'
        print(msg)
        wandb.log({f"{mode} Loss": running_loss/epoch_samples, f"{mode} Categorical Accuracy":cat_acc})
        plot_confusion_matrix(np.concatenate(lab), np.concatenate(pred), [i for i in range(5)])
        plot_heatmap(model)
        conf = cv2.imread('./confusion_matrix.png', cv2.IMREAD_COLOR)
        conf = cv2.cvtColor(conf, cv2.COLOR_BGR2RGB)
        wandb.log({"Confusion_Matrix": [wandb.Image(conf, caption="Confusion Matrix")]})
        heatmap = cv2.imread('./heatmap.png', cv2.IMREAD_COLOR)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        wandb.log({"Class Activation Mapping": [wandb.Image(heatmap, caption="Heatmap")]})
        history.loc[epoch, f'{mode}_loss'] = running_loss/epoch_samples
        history.loc[epoch, f'{mode}_cat_acc'] = cat_acc
        # NaN check
        if running_loss/epoch_samples > loss_thr or running_loss!=running_loss:
            print('\033[91mMixed Precision\033[0m rendering nan value. Forcing \033[91mMixed Precision\033[0m to be False ...')
            m_p = False
            batch_size = batch_size//2
            accum_step = accum_step*2
            print('Loading last best model ...')
            tmp = torch.load(os.path.join(model_dir, model_name+'_loss.pth'))
            model.load_state_dict(tmp['model'])
            optimizer.load_state_dict(tmp['optim'])
            lr_reduce_scheduler.load_state_dict(tmp['scheduler'])
            del tmp
            
        if record:
            history.to_csv(f'{history_dir}/history_{model_name}_{img_dim}.csv', index=False)
        return running_loss/epoch_samples, cat_acc


plist = [ 
        {'params': model.backbone.parameters(),  'lr': learning_rate/100},
        {'params': model.output.parameters(),  'lr': learning_rate}
    ]
optimizer = optim.Adam(plist, lr=learning_rate)
lr_reduce_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=patience, verbose=True, threshold=1e-4, threshold_mode='rel', cooldown=0, min_lr=1e-7, eps=1e-08)

In [ ]:
def main():
  prev_epoch_num = 0
  best_valid_loss = np.inf
  best_valid_acc = 0.0

  if load_model:
    tmp = torch.load(os.path.join(model_dir, model_name+'_acc.pth'))
    model.load_state_dict(tmp['model'])
    optimizer.load_state_dict(tmp['optim'])
    lr_reduce_scheduler.load_state_dict(tmp['scheduler'])
    scaler.load_state_dict(tmp['scaler'])
    prev_epoch_num = tmp['epoch']
    best_valid_loss = tmp['best_loss']
    best_valid_loss, best_valid_acc = train_val(prev_epoch_num+1, valid_loader, optimizer=optimizer, train=False, mode='val')
    del tmp
    print('Model Loaded!')
  
  for epoch in range(prev_epoch_num, n_epochs):
    torch.cuda.empty_cache()
    print(gc.collect())

    train_val(epoch, train_loader, optimizer=optimizer, train=True, mode='train')
    valid_loss, valid_acc = train_val(epoch, valid_loader, optimizer=optimizer, train=False, mode='val')
    print("#"*20)
    print(f"Epoch {epoch} Report:")
    print(f"Validation Loss: {valid_loss :.4f} Validation ACC: {valid_acc :.4f}")
    best_state = {'model': model.state_dict(), 'optim': optimizer.state_dict(), 'scheduler':lr_reduce_scheduler.state_dict(), 
          'scaler': scaler.state_dict(),
    'best_loss':valid_loss, 'best_acc':valid_acc, 'epoch':epoch}
    best_valid_loss, best_valid_acc = save_model(valid_loss, valid_acc, best_valid_loss, best_valid_acc, best_state, os.path.join(model_dir, model_name))
    print("#"*20)
   
if __name__== '__main__':
  main()